### Hydrofabric Builds MVP

This cookbook is an MVP to show a build procedure for getting from the v3 Reference Fabric to a hydrofabric product with Nexus, Flowpath, Divide, and Network Layers

The data files in this notebook are preprocessed v3 reference flowpaths and divides that are clipped to a HUC12 basin (010600010202) at Crystal Lake-Collyer Brook

In [ ]:
# import all necessary modules
import geopandas as gpd

from examples.cookbook.workflow import (
    aggregate_geometries_from_pairs_and_groups,
    aggregate_with_all_rules,
    build_hydroseq_network,
    find_outlets_by_hydroseq,
    reindex_layers_with_topology,
)

In [ ]:
# load all references used in this proof of concept
huc_gdf = gpd.read_file("sample_huc.gpkg")
fp_gdf = gpd.read_file("sample_flowpaths.gpkg")
div_gdf = gpd.read_file("sample_divides.gpkg")

In [ ]:
# let's plot the outputs to visualize the outputs of our reference
m = huc_gdf.explore(color="black", fill=False, weight=5)
m = div_gdf.explore(m=m, color="red", alpha=0.2)
fp_gdf.explore(m=m, color="blue")

### Step 1: Create a Network Structure

First we will need to determine individual networks within our reference and construct a graph oriented object to connect flowpaths to their upstream neighbors

In [ ]:
network = build_hydroseq_network(fp_gdf)
outlets = find_outlets_by_hydroseq(fp_gdf)

print(f"Found {len(outlets)} outlets")
print(f"Outlet: {outlets}")

print("Network Object:")
print(network)

### Step 2: Refactoring of the network to determine divides to be aggregated

Once the graph structure is created for the reference flowpaths, small flowpaths and divides need to be aggregated to better support routing stability. Currently, we are enforcing the following rules when aggreagating flowpaths:

#### Reference flowpath network consistency assumptions
- Hydroseq Reliability: The code assumes hydroseq/dnhydroseq fields provide accurate network topology. Lower hydroseq values indicate more downstream position, with differences reflecting true flow direction.
- Complete Network Coverage: All flowpaths in the dataset form a connected network graph. Missing or invalid dnhydroseq values indicate either outlets or data quality issues.

### Aggregation Rules
- Rule Priority Order: Algorithm applies rules in strict sequence: (1) 4km segment length Independence, (2) Small Catchment Aggregation ( < 0.1km2), (3) Stream Order-1 Branch Aggregation, (4) Drainage Area Aggregation. First applicable rule takes precedence.
- Length Threshold Absolute: Flowpaths ≥4km length ALWAYS remain independent, regardless of other characteristics. This represents a minimum modeling unit size requirement.
- Small Catchment Definition: flowpaths with areasqkm_left <0.1km² are considered "small catchments" requiring special aggregation treatment to prevent loss of hydrologic significance.
- Stream Order Hierarchy: Order-2 streams are preferred aggregation targets over Order-1 when available. Order-1 streams aggregate their entire upstream branch as a headwater unit.
- Drainage Area Dominance: When no order-specific rules apply, flowpaths aggregate with their largest upstream tributary by total drainage area (totdasqkm).

In [ ]:
segment_length_threshold = 4.0
small_catchment_threshold = 0.1
all_aggregation_pairs = []
all_headwater_groups = []
all_independent_flowpaths = []
all_minor_flowpaths = []

for outlet in outlets:
    print(f"  Processing outlet {outlet}...")
    result = aggregate_with_all_rules(
        network_graph=network,
        fp=fp_gdf,
        start_id=outlet,
        segment_length_threshold=segment_length_threshold,
        small_catchment_threshold=small_catchment_threshold,
    )

    all_aggregation_pairs.extend(result["aggregation_pairs"])
    all_headwater_groups.extend(result["headwater_groups"])
    all_independent_flowpaths.extend(result["independent_flowpaths"])
    all_minor_flowpaths.extend(result["minor_flowpaths"])

print("\nTotal aggregation relationships identified:")
print(f"  Pairs: {len(all_aggregation_pairs)}")
print(f"  Headwater groups: {len(all_headwater_groups)}")
print(f"  Independent: {len(all_independent_flowpaths)}")
print(f"  Flowlines: {len(all_minor_flowpaths)}")

### Step 3: Aggregate Geometries 

This step takes any flowpath items that need to be combined and aggregates their geometries together into a single shape. This is important for divides with zonal statistic calculations

In [ ]:
geometry_result = aggregate_geometries_from_pairs_and_groups(
    flowpaths_gdf=fp_gdf,
    divides_gdf=div_gdf,
    aggregation_pairs=all_aggregation_pairs,
    headwater_groups=all_headwater_groups,
    independent_flowpaths=all_independent_flowpaths,
    minor_flowpaths=all_minor_flowpaths,
)

### Step 4: Re-indexing flowpaths and divides and add nexus creation

Once geometries are reaggregated, a nexus-topology and re-indexing has to be performed to connect catchments -> flowpaths as a 1:1 reference and creates nexus points for flow aggregation. The network table is also created.

*NOTE:* Flowlines (which we call minor flowpaths to reduce confusion) are not included in the flowpaths layer as the v3 prototype Hydrofabric layers in VPU01 did not have these flowlines included. This capability is meant to use the v3 prototype Hydrofabric as a reference

*NOTE:* NHD references through `hf_id` were not included as the NHD dataset is too large to fit in a notebook and ship with the code. 

In [ ]:
hf = reindex_layers_with_topology(fp_gdf, div_gdf, geometry_result)

### Let's look at our created Hydrofabric 

Now that we've run all of the end -> end steps, let's take a look at each of the required layers. The database schema of each layer is intended to match that used by the prototype v3 Hydrofabric

In [ ]:
# network
hf["network"].head()

In [ ]:
# divides
hf["divides"].head()

In [ ]:
# flowpaths
hf["flowpaths"].head()

In [ ]:
# nexus
hf["nexus"].head()

In [ ]:
# Let's view the outputs on a map
m = huc_gdf.explore(color="black", fill=False, weight=5)
m = hf["divides"].explore(m=m, color="red", alpha=0.2)
m = hf["flowpaths"].explore(m=m, color="blue")
hf["nexus"].explore(m=m, color="black")

In [ ]:
# We can also save the outputs to disk for external verification if necessary
output_file = "MVP_NGWPC_hydrofabric.gpkg"
for table_name, _layer in hf.items():
    if len(_layer) > 0:
        gpd.GeoDataFrame(_layer).to_file(output_file, layer=table_name, driver="GPKG")
    else:
        print(f"Warning: {table_name} layer is empty")